# Building second level models using _nipype_ and _SPM12_

## Template for _megameta_ project

-------
#### History

* 07/06/20 nc - adding tasks
* 12/19 jahn - tailored for individual bhv change analysis
* 11/6/19 mbod, jeesung, ncooper - testing pipeline code with new individual diff measure to replicate MM report models
* 5/29/19 cscholz - use subject level mreg split up into aversive/nonaversive image projects
* 5/8/19 mbod & bdore - use subject level mreg
* 3/18/19 mbod  - modify example notebook to make template

-----

### Description

* Set up a nipype workflow to use SPM12 to make second (group) level models for _megameta_ task data (preprocessed using `batch8` SPM8 scripts) in BIDS derivative format
* Includes both single sample t-test and multiple regression designs


In [1]:
%matplotlib inline
import os
import json
import nibabel as nb

-------------------

### Step 1: SET NEEDED PARAMETERS

* Specify the following values:
    1. project name - should be name of folder under `/data00/project/megameta`, e.g. `project1`
    2. filename for JSON model specification (should be inside `model_specification` folder), e.g. `p1_image_pmod_likeme.json`
    3. TR value in seconds
 



In [2]:
MODEL_SPEC_FILE = 'group_MESSAGE_mreg_behav_7studies.json'
contrast_name='puremessage'

In [3]:
PATH_TO_SPM_FOLDER = '/data00/tools/spm12mega/'

In [4]:
ROOT_DIR = '/data00/projects/megameta'
GROUP_DIR = '/data00/projects/megameta/group_models/'

fwhm = [ 8 ] # smoothing kernels
resolutions = ['medium']

cont1 = ['Change', 'T', ['change','baseline','mean'], [1,0,0]]
#cont2 = ['Change', 'T', ['change','baseline','mean'], [0,1,0]]
#cont3 = ['Group', 'T', ['postmessage','baseline','mean'], [0,0,1]]


contrast_list=[cont1]

In [20]:
JSON_MODEL_FILE = os.path.join('/data00/projects/megameta/scripts/jupyter_megameta/second_level_models',
                               'model_specifications',
                               MODEL_SPEC_FILE)

with open(JSON_MODEL_FILE) as fh:
    model_def = json.load(fh)
    
MODEL_NAME = model_def['ModelName']

mask_path='/data00/tools/spm8/apriori/brainmask_th25.nii'
mask_name='th25'

mask = nb.load(mask_path)

output_dir = os.path.join(GROUP_DIR,'derivatives', 'nipype','model_2nd-level_{}'.format(MODEL_NAME),contrast_name,'change')        
working_dir = os.path.join(GROUP_DIR, 'working','nipype','workingdir_model_2nd-level_{}'.format(MODEL_NAME),contrast_name,'change')   

if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(['creating output directory:',output_dir])
if not os.path.exists(working_dir):
    os.makedirs(working_dir)
    print(['creating working directory',working_dir])

* Eligible subjects list (`subjects_list`) is created from checking the `func` folders in BIDS project and finding matching NIFTI files for task.


* You can edit subjects to be included in processing through:
    1. making a list of subject ids to be excluded in `exclude_subjects` list
        - these subjects will be removed from the list of eligible subjects
    2. specifying a list of subject ids to be included in processing in `include_subjects` list
        - the subjects in this list will replace the eligible subjects
    

In [21]:
exclude_subjects=[
'sub-AS16', 'sub-AS17', 'sub-AS35', 'sub-AS42','sub-AS53','sub-AS58',
'sub-P102','sub-P104','sub-P110','sub-P117','sub-P123','sub-P133','sub-P152',
'sub-HA01', 
'sub-HA02',
'sub-HA03',
'sub-HA04',
'sub-HA05',
'sub-HA06',
'sub-HA09',
'sub-HA10',
'sub-HA11',
'sub-HA12',
'sub-HA13',
'sub-HA14',
'sub-HA15',
'sub-HA16',
'sub-HA33',
'sub-HA58',
'sub-HA67',
'sub-HA07',
'sub-HA08',
'sub-HA17',
'sub-HA19',
'sub-HA38',
'sub-HA43',
'sub-HA52',
'sub-HA56',
'sub-HA59',
'sub-PA057',
'sub-PA214',
'sub-PA043',
'sub-PA001',
'sub-PA002',
'sub-PA004',
'sub-PA013',
'sub-PA015',
'sub-PA022',
'sub-PA023',
'sub-PA024',
'sub-PA026',
'sub-PA030',
'sub-PA033',
'sub-PA039',
'sub-PA042',
'sub-PA043',
'sub-PA044',
'sub-PA045',
'sub-PA051',
'sub-PA053',
'sub-PA059',
'sub-PA062',
'sub-PA064',
'sub-PA066',
'sub-PA069',
'sub-PA070',
'sub-PA072',
'sub-PA073',
'sub-PA075',
'sub-PA076',
'sub-PA078',
'sub-PA079',
'sub-PA081',
'sub-PA083',
'sub-PA087',
'sub-PA091',
'sub-PA092',
'sub-PA096',
'sub-PA097',
'sub-PA099',
'sub-PA104',
'sub-PA105',
'sub-PA106',
'sub-PA110',
'sub-PA115',
'sub-PA117',
'sub-PA118',
'sub-PA119',
'sub-PA121',
'sub-PA123',
'sub-PA124',
'sub-PA127',
'sub-PA128',
'sub-PA129',
'sub-PA130',
'sub-PA131',
'sub-PA134',
'sub-PA138',
'sub-PA139',
'sub-PA140',
'sub-PA151',
'sub-PA152',
'sub-PA155',
'sub-PA161',
'sub-PA168',
'sub-PA171',
'sub-PA178',
'sub-PA184',
'sub-PA186',
'sub-PA187',
'sub-PA190',
'sub-PA193',
'sub-PA195',
'sub-PA198',
'sub-PA199',
'sub-PA203',
'sub-PA204',
'sub-PA205',
'sub-PA211',
'sub-PA215',
'sub-PA216',
'sub-PA217',
'sub-PA218',
'sub-BA224',
'sub-BA228',
'sub-BA233',
'sub-BA239',
'sub-BA267',
'sub-BA272',
'sub-BA276',
'sub-WA110',
'sub-WA204'] 

In [22]:
GROUP_DIR = '/data00/projects/megameta/group_models/'


-------------------------

### Step 2: RUN FIRST LEVEL SETUP NOTEBOOK

In [23]:
%run "/data00/projects/megameta/scripts/jupyter_megameta/cnlab/GLM/second_level_mreg_Jeesung.ipynb"    

-------------------------

### Step 3: CHECK WORKFLOW AND DIRECTORIES

In [24]:
print('Working directory for pipeline is:\n\t',working_dir)

print('\n\nFinal model directory for pipeline is:\n\t',output_dir)

#print('\n\nThese subjects will be processed:\n\t', ' '.join(subject_list))

Working directory for pipeline is:
	 /data00/projects/megameta/group_models/working/nipype/workingdir_model_2nd-level_MESSAGE_mreg_behav_7studies/puremessage/change


Final model directory for pipeline is:
	 /data00/projects/megameta/group_models/derivatives/nipype/model_2nd-level_MESSAGE_mreg_behav_7studies/puremessage/change


-------------------------

### Step 4: RUN PIPELINE

In [27]:
run_graph=l2analysis.run('Linear') # find participants who both have 1st level models & behavioral measures

200706-16:17:41,972 nipype.workflow INFO:
	 Workflow l2analysis settings: ['check', 'execution', 'logging', 'monitoring']
200706-16:17:42,19 nipype.workflow INFO:
	 Running serially.
200706-16:17:42,21 nipype.workflow INFO:
	 [Node] Setting-up "l2analysis.makecontrasts" in "/fmriNASTest/data00/projects/megameta/group_models/working/nipype/workingdir_model_2nd-level_MESSAGE_mreg_behav_7studies/puremessage/change/l2analysis/makecontrasts".
200706-16:17:42,301 nipype.workflow INFO:
	 [Node] Running "makecontrasts" ("nipype.interfaces.utility.wrappers.Function")
excluded subject list:  ['sub-AS16', 'sub-AS17', 'sub-AS35', 'sub-AS42', 'sub-AS53', 'sub-AS58', 'sub-P102', 'sub-P104', 'sub-P110', 'sub-P117', 'sub-P123', 'sub-P133', 'sub-P152', 'sub-HA01', 'sub-HA02', 'sub-HA03', 'sub-HA04', 'sub-HA05', 'sub-HA06', 'sub-HA09', 'sub-HA10', 'sub-HA11', 'sub-HA12', 'sub-HA13', 'sub-HA14', 'sub-HA15', 'sub-HA16', 'sub-HA33', 'sub-HA58', 'sub-HA67', 'sub-HA07', 'sub-HA08', 'sub-HA17', 'sub-HA19', 's